In [ ]:
import numpy as np
import tensorflow as tf
import os
import gym
import scipy.signal # for cumulative returns
from numpy.linalg import inv # for conjugate gradient

In [ ]:
# PARAMETERS

env = gym.make('Acrobot-v1')
state_size = env.observation_space.shape
action_size = env.action_space.shape

eps = 1e-8
total_episodes = 30
gamma = 0.99
max_path_len = 2000
total_steps = 20000
cg_damping = 0.1
max_kl = 0.01

In [ ]:
# FUNCTIONS

def gauss_log_prob(x, mu, logstd):
    var = tf.math.exp(2 * logstd)
    prob = -tf.math.square(x - mu) / (2 * var) - 0.5 * tf.math.log(tf.constant(2 * np.pi)) - logstd
    return tf.math.reduce_sum(prob, axis = 1)


def gauss_kl(mu1, logstd1, mu2, logstd2):
    var1 = tf.math.exp(2 * logstd1)
    var2 = tf.math.exp(2 * logstd2)
    return tf.math.reduce_sum(logstd2 - logstd1 + (var1 + tf.math.square(mu1 - mu2)) / (2 * var2) - 0.5)


def gauss_entropy(mu, logstd):
    return tf.math.reduce_sum(logstd + 0.5 * tf.constant(2 * np.pi * np.e, dtype = tf.float32))


def numel(x):
    return np.prod(var_shape(x))


def flat_grad(loss, var_list):
    grads = tf.gradients(loss, var_list)
    return tf.concat(values=[tf.reshape(grad, [numel(v)]) for (v, grad) in zip(var_list, grads)], axis=0)


def gauss_kl_firstfixed(mu, logstd):
    mu1, logstd1 = map(tf.stod_gradient, [mu, logstd])
    
    return gauss_kl(mu1, logstd1, mu, logstd)


def slice_vector(vector, shapes):
    start = 0
    tensors = []
    for shape in shapes:
        size = np.prod(shape)
        tensor = tf.reshape(vector[start:(start + size)], shape)
        tensors.append(tensor)
        start += size
    return tensors


def get_cumulative_returns(r, gamma):
    r = np.array(r)
    return scipy.signal.lfilter([1], [1, -gamma], r[::-1], axis=0)[::-1]


class DistributionInit:
    def __init__(self, filter_mean=True):
        self.m1 = 0
        self.v = 0
        self.n = 0.
        self.filter_mean = filter_mean

    def __call__(self, o):
        self.m1 = self.m1 * (self.n / (self.n + 1)) + o    * 1/(1 + self.n)
        self.v = self.v * (self.n / (self.n + 1)) + (o - self.m1)**2 * 1/(1 + self.n)
        self.std = (self.v + 1e-6)**.5 # std
        self.n += 1
        if self.filter_mean: 
            o1 =  (o - self.m1)/self.std
        else:
            o1 =  o/self.std
        o1 = (o1 > 10) * 10 + (o1 < -10)* (-10) + (o1 < 10) * (o1 > -10) * o1 
        return o1


dist_init = DistributionInit()


def rollout(env, agent, max_path_len, total_steps, gamma):
    steps_count = 0
    paths = []
    
    while steps_count < total_steps:
        observations, actions, rewards, action_dists_mu, action_dists_logstd = [], [], [], [], []
        observation = dist_init(env.reset())
        
        for _ in range(max_path_len):
            steps_count += 1
            action, policy = agent.act(observation)
            
            observations.append(observation)
            actions.append(actions)
            action_dists_mu.append(policy.get('action_dist_mu', default=[]))
            action_dists_logstd.append(policy.get('action_dist_logstd', default=[]))
            
            observation, reward, done, _ = env.step(action)
            
            observation = dist_init(observation)
            rewards.append(reward)
            
            if done or steps_count == total_steps:
                path = {'observations' : np.concatenate(np.expand_dims(obs, axis=0)),
                        'action_dists_mu' : np.concatenate(action_dists_mu),
                        'action_dists_logstd' : np.concatenate(action_dists_logstd),
                        'rewards' : np.array(rewards),
                        'actions' :  np.array(actions), 
                        'cumulative_returns' : get_cumulative_returns(rewards, gamma)
                       }
                paths.append(path)
                break
   
    return paths


class FlatWeights:
    def __init__(self, sess, var_list):
        self.sess = sess
        self.flat = tf.concat([tf.reshape(v, shape=[numel(v)]) for v in var_list], axis=0)
        
    def __call__(self):
        return self.flat.eval(session=self.sess)


class LoadFlatWeights:
    def __init__(self, sess, var_list):
        self.sess = sess
        assigns = []
        var_shape = [var.value for var in var_list.get_shape()]
        shapes = map(var_shape, var_list)
        total_size = sum(np.prod(shape) for shape in shapes)
        self.theta = tf.placeholder(dtype=tf.float32, shape=[total_size])
        
        theta = self.weights
        start = 0
        for (shape, v) in zip(shapes, var_list):
            size = np.prod(shape)
            assigns.append(tf.assign(v, tf.reshape(theta[start:start + size], shape)))
            start += size
        
        self.op = tf.group(*assigns)
        
    
    def __call__(self, theta):
        self.sess.run(self.op, feed_dict={self.theta : theta})

    
def conjugate_gradient(f_Ax, b, cg_iters=10, residual_tol=1e-10):
    p = b.copy()
    r = b.copy()
    x = np.zeros_like(b)
    rdotr = r.dot(r)
    for i in range(cg_iters):
        z = f_Ax(p)
        v = rdotr / (p.dot(z) + 1e-8)
        x += v * p
        r -= v * z
        newrdotr = r.dot(r)
        mu = newrdotr / (rdotr + 1e-8)
        p = r + mu * p
        rdotr = newrdotr
        if rdotr < residual_tol:
            break
    return x


def linesearch(f, x, full_step, expected_improve_ratio):
    max_backtracks = 10
    accept_ratio = 0.1
    loss, _, _ = f(x)
    
    for (_, step_frac) in enumerate((0.5)**(np.arange(max_backtrack))):
        new_x = x + step_frac * full_step
        new_loss = f(new_x)
        actual_improve = loss - new_loss
        expected_improve = expected_improve_ratio * step_frac
        ratio = actual_improve / expected_improve
        
        if ratio > accept_ratio and actual_improve > 0:
            return new_x
        
    return x
    

In [ ]:
class Agent:
    def __init__(self, env, state_size, action_size, eps, total_episodes, gamma, max_path_len, total_steps, cg_damping=0.1, max_kl=0.01):
        self.env = env
        self.state_size = state_size[0]
        self.action_size = np.prod(action_size)
        self.sess = tf.Session()
        self.eps = eps
        self.total_episodes = total_episodes
        self.gamma = gamma
        self.max_path_len = max_path_len
        self.total_steps = total_steps
        self.cg_damping = cg_damping
        self.max_kl = max_kl
        
        self.obs = tf.placeholder(dtype=tf.float32, shape=[None, state_size], name='state')
        self.action = tf.placeholder(dtype=tf.float32, shape=[None, action_size], name='action')
        self.advantage = tf.placeholder(dtype=tf.float32, shape=[None], name='advantage')
        
        # Gauss parameters
        self.old_action_dist_mu = tf.placeholder(dtype=tf.float32, shape=[None, action_size])
        self.old_action_dist_logstd = tf.placeholder(dtype=tf.float32, shape=[None, action_size])
        
        # Neural network
        self.first_layer = tf.contrib.layers.fully_connected(inputs=self.obs, num_outputs=64, name='first_layer')
        self.second_layer = tf.contrib.layers.fully_connected(inputs=self.first_layer, num_outputs=64, name='second_layer')
        self.output = tf.contrib.layers.fully_connected(inputs=self.second_layer, num_outputs=self.action_size, name='output')
        
        self.action_dist_logstd_param = tf.Variable((0.01 * np.random.randn(1, self.action_size)).astype(np.float32), name='action_dist_logsd_param')
        self.action_dist_logstd = tf.tile(self.action_dist_logstd_param, tf.stack(tf.shape(self.output)[0], 1))
        
        log_prob = gauss_log_prob(self.action, self.action_dist_mu, self.action_dist_logstd)
        log_prob_old = gauss_log_prob(self.action, self.old_action_dist_mu, self.old_action_dist_logstd)
        
        # L_surr = -E[ prob / prob_old * advantage ]
        frac = tf.exp(log_prob - log_prob_old)
        L_surr = -tf.math.reduce_mean(frac * self.advantage)
        
        var_list = tf.trainable_variables()
        
        batch_size = tf.cast(tf.shape(self.state_size)[0], dtype=tf.float32)
        kl = gauss_kl(self.old_action_dist_mu, self.old_action_dist_logstd, self.output, self.action_dist_logstd) / batch_size
        entropy = gauss_entropy(self.output, self.action_dist_logstd) / batch_size
        
        self.losses = [L_surr, kl, entropy]
        
        self.flat_grad_surr = flat_grad(L_surr, var_list)
        
        kl_firstfixed = gauss_kl_firstfidex(self.output, self.action_dist_logstd) / batch_size
        grads = tf.gradients(kl_firstfixed, var_list)
        
        self.flat_tangent = tf.placeholder(dtype=tf.float32, shape=[None])
        var_shape = [var.value for var in var_list.get_shape()]
        shapes = map(var_shape, var_list)
        tangents = slice_vector(self.flat_tangent, shapes)
        
        grad_vec_prod = [tf.math.reduce_sum(g * t) for (g, t) in zip(grads, tangents)]
        self.fisher_vec_prod = flat_grad(grad_vec_prod, var_list)
        
        self.flat_weights = FlatWeights(self.sess, var_list)
        self.load_flat_weights = LoadFlatWeights(self.sess, var_list)
        self.sess.run(tf.initialize_variables(var_list))
        
        
        
    def act(self, obs):
        obs = np.expand_dims(obs, axis=0)
        action_dist_mu, action_dist_logstd = self.sess.run([self.output, self.action_dist_logstd], feed_dict={self.obs : obs})
        action = action_dist_mu + np.exp(action_dist_logstd) * np.random.randn(*action_dist_logstd.shape)
        
        return action.ravel(), {'action_dist_mu' : action_dist_mu, 'action_dist_logstd' : action_dist_logstd}
        
        
    def learn(self, ):
        episode_count = 0
        
        for i in range(self.total_episodes):
            paths = rollout(self.env, self, self.max_path_len, self.total_steps, self.gamma)
            
            
            # TODO: compute advantages
            
            
            # Updating policy
            observations = np.concatenate([path['observations'] for path in paths])
            actions = np.concatenate([path['actions'] for path in paths])
            action_dist_mu = np.concatenate([path['action_dists_mu'] for path in paths])
            action_dist_logstd = np.concatenate([path['action_dists_logstd'] for path in paths])
            
            # Normalizing advantages
            advantages = np.concatenate([path['advantages'] for path in paths])
            advantages -= advantages.mean()
            advantages /= (advantages.std() + 1e-8)
            
            self.vf.fit(paths)
            
            feed_dict = {self.obs : observations, 
                            self.action : actions,
                            self.advantage : advantages, 
                            self.old_action_dist_mu : action_dist_mu, 
                            self.old_action_dist_logstd : action_dist_logstd
                        }
            
            prev_theta = self.flat_weights()
            
            def fisher_vector_product(p):
                feed_dict[self.flat_tangent] = p
                return self.sess.run(self.fisher_vec_prod, feed_dict=feed_dict) + p * cg_damping
            
            flat_grad = self.sess.run(self.flat_grad_surr, feed_dict=feed_dict)
            
            step_dir = conjugate_gradient(fisher_vector_product, -flat_grad)
            
            shs = (0.5 * step_dir.dot(fisher_vector_product(step_dir)))
            
            lm = np.sqrt(shs / self.max_kl)
            
            full_step = step_dir / lm
            
            def loss(theta):
                self.load_flat_weights(theta)
                return self.sess.run(self.losses[0], feed_dict=feed_dict)
            
            new_theta = linesearch(loss, prev_theta, full_step, -flat_grad.dot(stepdir) / lm)
            new_theta = prev_theta + full_step
            
            self.load_flat_weights(new_theta)
            
            # TODO: manage graphs, visualization
            
            # Report progress
            L_surr, kl, entropy = self.sess.run(self.losses, feed_dict=feed_dict)
            episode_rewards = np.array([path['rewards'] for path in paths])
            episode_count += len(episode_rewards)
            
            stats = {}
            stats['Total number of episodes'] = episode_count
            stats['Average sum of rewards per episode'] = episode_rewards.mean()
            stats['Entropy'] = entropy
            stats['KL between old and new distribution'] = kl
            stats['Surrogate loss'] = L_surr
            print('\n-------------------- Iteration {} --------------------'.format(i))
            for k, v in stats.iteritems():
                print(k + ': ' + ' ' * (40 - len(k)) + str(v))
            
            if entropy != entropy:
                exit(-1)
            

In [ ]:
agent = Agent(env, state_size, action_size, eps, total_episodes, gamma, max_path_len, total_steps, cg_damping, max_kl)
agent.learn()

# TODO: run TRPO